In [1]:
import requests
import bs4
import re
import xlsxwriter
import nltk
from nltk.tokenize import word_tokenize

In [2]:
npi_list=['pv_npi']
fname_list=['pv_fname']
lname_list=['pv_lname']
add1_list=['pv_add1']
city_list=['pv_city']
state_list=['pv_state']
zipcode_list=['pv_zip']

In [16]:
npi = ['1033155555']

for i in npi:
    print(i)
    res = requests.get('https://npiregistry.cms.hhs.gov/registry/provider-view/'+i)
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    name_text = soup.select('p')
    name_text[0].getText()
    name_token = word_tokenize(name_text[0].getText())
    stop_words=['MMS','PH.D','MRS.','INCORPORATED','N.P','MR.','DPM','RN, CFNP','D.O','O.D','PA-C','MD','MS.','.','DR.','M.D','JR.','RN', ',', 'CFNP','MEDICAL','DOCTOR','F.A.C.P','M.D.']
    name_token_cleaned = [word for word in name_token if not word in stop_words]
    stop_words2=[',']
    name_token_cleaned2 = [word for word in name_token if not word in stop_words2]
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    add_chunk = soup.select('td')
    
    facility_word = ['INC','SYSTEMS','CORPORATION','PROTRANSPORT','HEALTH','HOSPITAL','CENTER','GROUP','INCORPORATED']
    if any(c in facility_word for c in name_token):
        fname=''
        lname=' '.join(name_token_cleaned2)
        add_text = add_chunk[11].getText()
    else:
        fname=name_token_cleaned[0].replace('-',' ')
        lname=name_token_cleaned[len(name_token_cleaned)-1]
        add_text = add_chunk[13].getText()
    print("fname: "+fname)
    print("lname: "+lname)
         
    add_text_cleaned = add_text.replace("\n", "~").replace("\t", "")
   
    if "DIVISION OF HOSPITAL MEDICINE, UCSF" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("DIVISION OF HOSPITAL MEDICINE, UCSF", "")         
    elif "2ND FLOOR" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("2ND FLOOR", "")
    elif "# M-391" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("# M-391", "")
    elif "MB-2" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("MB-2", "")
    elif "MC #9116A" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("MC #9116A", "")          
    elif "LOWER LEVEL" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("LOWER LEVEL", "") 
    elif "ATTN. HANFORD MEDICAL ASSOCIATES" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("ATTN. HANFORD MEDICAL ASSOCIATES", "")    
    elif "BETH ISRAEL DEACONESS MEDICALCTR.~330 BROOKLINE AVE., PBS 2ND FLOOR" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("BETH ISRAEL DEACONESS MEDICALCTR.~330 BROOKLINE AVE., PBS 2ND FLOOR", "330 BROOKLINE AVE")    
    elif "PATHOLOGY LAB DEPT" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("PATHOLOGY LAB DEPT", "")
    elif "S1-10" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("S1-10", "")
        
    else:
        add_text_cleaned2=add_text_cleaned
        
    if "STE" in add_text_cleaned2:
        rmv_reg = re.compile("(STE[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "SUITE" in add_text_cleaned2:
        rmv_reg = re.compile("(SUITE[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "ROOM" in add_text_cleaned2:
        rmv_reg = re.compile("(ROOM[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "BOX" in add_text_cleaned2:
        rmv_reg = re.compile("(BOX[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "~FL" in add_text_cleaned2:
        rmv_reg = re.compile("(FL[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"") 
    elif "#" in add_text_cleaned2:
        rmv_reg = re.compile("(#[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "RM" in add_text_cleaned2:
        rmv_reg = re.compile("(RM[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "LEVEL" in add_text_cleaned2:
        rmv_reg = re.compile("(LEVEL[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "CEP" in add_text_cleaned2:
        rmv_reg = re.compile("(CEP[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "APT" in add_text_cleaned2:
        rmv_reg = re.compile("(APT[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
        
    else:
        add_text_cleaned3=add_text_cleaned2       
        
    regexp = re.compile("[~]+\s+([\d\-]+\s+[\w\s]+[\w]+)[\,\.\s\d\~]*(\w[\w\s]+)\,\s([\w]{2})\s+[~]+([\d]{5})")
    add1 = regexp.search(add_text_cleaned3).group(1)
    city = regexp.search(add_text_cleaned3).group(2)
    state = regexp.search(add_text_cleaned3).group(3)
    zipcode = regexp.search(add_text_cleaned3).group(4)
    print("add1: "+add1)
    print("city: "+city)
    print("state: "+state)
    print("zip: "+zipcode)
    npi_list.append(i)
    fname_list.append(fname)
    lname_list.append(lname)
    add1_list.append(add1)
    city_list.append(city)
    state_list.append(state)
    zipcode_list.append(zipcode)

1033155213
fname: EDWARD
lname: CHAN
add1: 728 PACIFIC AVE
city: SAN FRANCISCO
state: CA
zip: 94133
1184607970
fname: HANLON
lname: FONG
add1: 2000 VAN NESS AVE
city: SAN FRANCISCO
state: CA
zip: 94109
1235143702
fname: JEROME
lname: JEW
add1: 919 CLAY ST
city: SAN FRANCISCO
state: CA
zip: 94108
1235143702
fname: JEROME
lname: JEW
add1: 919 CLAY ST
city: SAN FRANCISCO
state: CA
zip: 94108
1265461644
fname: FRED
lname: LUI
add1: 728 PACIFIC AVE
city: SAN FRANCISCO
state: CA
zip: 94133
1265714281
fname: KATHERINE
lname: LEE
add1: 929 CLAY ST
city: SAN FRANCISCO
state: CA
zip: 94108
1265815955
fname: 
lname: CITYVIEW EYE CENTER A MEDICAL CORPORATION
add1: 929 CLAY ST
city: SAN FRANCISCO
state: CA
zip: 94108
1275532145
fname: SIMON
lname: LEE
add1: 1828 EL CAMINO REAL
city: BURLINGAME
state: CA
zip: 94010
1285737692
fname: HO
lname: TAN
add1: 890 JACKSON ST
city: SAN FRANCISCO
state: CA
zip: 94133
1306855564
fname: 
lname: ARTHUR S. CHIN M.D. INC .
add1: 770 STOCKTON ST
city: SAN FRANCISCO

In [17]:
workbook = xlsxwriter.Workbook('pv name.xlsx')
worksheet = workbook.add_worksheet()

array = [npi_list,fname_list,lname_list,add1_list,city_list,state_list,zipcode_list]

row = 0

for col, data in enumerate(array):
    worksheet.write_column(row, col, data)

workbook.close()

In [ ]:
#TEST

In [4]:
res = requests.get('https://npiregistry.cms.hhs.gov/registry/provider-view/1306855564')
#res.text

In [5]:
soup = bs4.BeautifulSoup(res.text, 'lxml')
name_text = soup.select('p')
name_text[0].getText()

'   \n                  ARTHUR S. CHIN, M.D., INC. '

In [8]:
name_token = word_tokenize(name_text[0].getText())
stop_words=['MMS','PH.D','MRS.','INCORPORATED','N.P','MR.','DPM','RN, CFNP','D.O','O.D','PA-C','MD','MS.','.','DR.','M.D','JR.','RN', ',', 'CFNP','MEDICAL','DOCTOR','F.A.C.P','M.D.']
name_token_cleaned = [word for word in name_token if not word in stop_words]
name_token_cleaned

['ARTHUR', 'S.', 'CHIN', 'INC']

In [9]:
stop_words2=[',']
name_token_cleaned2 = [word for word in name_token if not word in stop_words2]
name_token_cleaned2

['ARTHUR', 'S.', 'CHIN', 'M.D.', 'INC', '.']

In [10]:
facility_word = ['INC','SYSTEMS','CORPORATION','PROTRANSPORT','HEALTH','HOSPITAL','CENTER','GROUP','INCORPORATED']
if any(c in facility_word for c in name_token):
    fname=''
    lname=' '.join([word for word in name_token if not word in stop_words2])
    add_text = add_chunk[11].getText()
else:
    fname=name_token_cleaned[0].replace('-',' ')
    lname=name_token_cleaned[len(name_token_cleaned)-1]
    add_text = add_chunk[13].getText()
print("fname: "+fname)
print("lname: "+lname)

fname: 
lname: ARTHUR S. CHIN M.D. INC .


In [11]:
soup = bs4.BeautifulSoup(res.text, 'lxml')
add_chunk = soup.select('td')
add_text = add_chunk[11].getText()
add_text

' \n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n                                770 STOCKTON ST\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tSAN FRANCISCO, CA \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t94108-2313\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tUnited States\n\t\t\t\t\t\t\t\t\n                                Phone: 415-982-2292 | \n\t\t\t\t\t\t\t\tFax: 415-982-3910\n              \n\t\t\t\t\t\t\t\t\nView Map \n'

In [12]:
soup = bs4.BeautifulSoup(res.text, 'lxml')
add_chunk = soup.select('td')
if any(c in facility_word for c in name_token_cleaned):
    add_text = add_chunk[11].getText()
else:
    add_text = add_chunk[13].getText()
add_text


' \n\t\t\t\t\t\t\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n                                770 STOCKTON ST\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tSAN FRANCISCO, CA \n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t94108-2313\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tUnited States\n\t\t\t\t\t\t\t\t\n                                Phone: 415-982-2292 | \n\t\t\t\t\t\t\t\tFax: 415-982-3910\n              \n\t\t\t\t\t\t\t\t\nView Map \n'

In [13]:
add_text_cleaned = add_text.replace("\n", "~").replace("\t", "")
add_text_cleaned

' ~~~~                                770 STOCKTON ST~~~SAN FRANCISCO, CA ~~~94108-2313~~~~United States~~                                Phone: 415-982-2292 | ~Fax: 415-982-3910~              ~~View Map ~'

In [14]:
    if "DIVISION OF HOSPITAL MEDICINE, UCSF" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("DIVISION OF HOSPITAL MEDICINE, UCSF", "")         
    elif "2ND FLOOR" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("2ND FLOOR", "")
    elif "# M-391" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("# M-391", "")
    elif "MB-2" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("MB-2", "")
    elif "MC #9116A" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("MC #9116A", "")          
    elif "LOWER LEVEL" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("LOWER LEVEL", "") 
    elif "ATTN. HANFORD MEDICAL ASSOCIATES" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("ATTN. HANFORD MEDICAL ASSOCIATES", "")    
    elif "BETH ISRAEL DEACONESS MEDICALCTR.~330 BROOKLINE AVE., PBS 2ND FLOOR" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("BETH ISRAEL DEACONESS MEDICALCTR.~330 BROOKLINE AVE., PBS 2ND FLOOR", "330 BROOKLINE AVE")    
    elif "PATHOLOGY LAB DEPT" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("PATHOLOGY LAB DEPT", "")
    elif "S1-10" in add_text_cleaned:
        add_text_cleaned2 = add_text_cleaned.replace("S1-10", "")
    
    else:
        add_text_cleaned2=add_text_cleaned
        
    if "STE" in add_text_cleaned2:
        rmv_reg = re.compile("(STE[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "SUITE" in add_text_cleaned2:
        rmv_reg = re.compile("(SUITE[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "ROOM" in add_text_cleaned2:
        rmv_reg = re.compile("(ROOM[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "BOX" in add_text_cleaned2:
        rmv_reg = re.compile("(BOX[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "~FL" in add_text_cleaned2:
        rmv_reg = re.compile("(FL[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"") 
    elif "#" in add_text_cleaned2:
        rmv_reg = re.compile("(#[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "RM" in add_text_cleaned2:
        rmv_reg = re.compile("(RM[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "LEVEL" in add_text_cleaned2:
        rmv_reg = re.compile("(LEVEL[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "CEP" in add_text_cleaned2:
        rmv_reg = re.compile("(CEP[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
    elif "APT" in add_text_cleaned2:
        rmv_reg = re.compile("(APT[\w\d\-\s\#\.\,]+)\~")
        rmv = rmv_reg.search(add_text_cleaned2).group(0)
        add_text_cleaned3 = add_text_cleaned2.replace(rmv,"")
        
    else:
        add_text_cleaned3=add_text_cleaned2   
    add_text_cleaned3

' ~~~~                                770 STOCKTON ST~~~SAN FRANCISCO, CA ~~~94108-2313~~~~United States~~                                Phone: 415-982-2292 | ~Fax: 415-982-3910~              ~~View Map ~'

In [15]:
regexp = re.compile("[~]+\s+([\d\-]+\s+[\w\s]+[\w]+)[\,\.\s\d\~]*(\w[\w\s]+)\,\s([\w]{2})\s+[~]+([\d]{5})")
regexp.search(add_text_cleaned3).group(1)


'770 STOCKTON ST'